## Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import random
from pprint import pprint
warnings.filterwarnings('ignore')

## Import dataset


In [ ]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
df.rename(columns={'species': 'target'}, inplace=True)

In [ ]:
df.shape

(150, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


No missing values

In [ ]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


#### Feature Scaling not required in decision tree

## Train Test Split 

In [ ]:
def train_test_split(df, test_size):

  if isinstance(test_size, float):
    test_size = round(test_size*len(df))
  
  indices = df.index.tolist() # random.sample takes list, set, dictionary
  test_indices = random.sample(population=indices, k=test_size)

  test = df.loc[test_indices]
  train = df.drop(test_indices)

  return train, test

## Check purity of a node (Contains instance of only one class or not)

In [ ]:
def check_purity(data):
  unique_labels = np.unique(data[:, -1])
  if len(unique_labels) == 1:
    return True
  return False

## Classify
#### Define a function that decides the predicted class label in a leaf node based on maximum frequency

In [ ]:
def classify_data(data):

  unique, count_unique = np.unique(data[:, -1], return_counts=True)
  
  index = count_unique.argmax()
  classification = unique[index]

  return classification

## Define a potential splits function
Potential splits is a dictionary containing all the values where we split a node and check its entropy

For continuous value, find mean of two consecutive values (of 2 unique consecutive)

In [ ]:
def get_potential_splits(data):
    
    potential_splits = {}
    for column in range(data.shape[1] - 1):

        unique_values = np.unique(data[:, column])

        feature_type = FEATURE_TYPES
        if feature_type == 'continuous':  
          potential_splits[column] = []
          for index in range(len(unique_values)):

              if index != 0:
                  potential_split = (unique_values[index] + unique_values[index - 1]) / 2
                  potential_splits[column].append(potential_split)
    
        else:
          potential_splits[column] = unique_values

    return potential_splits

In [ ]:
# potential_splits = get_potential_splits(data)

# We get potential splits for all the columns

In [ ]:
# sns.scatterplot(data[:, 3], data[:, 2], hue=data[:,-1])

# plt.vlines(potential_splits[3], ymin=0, ymax=7)
# plt.show()

In [ ]:
# plt.figure(figsize=(10,10))
# sns.scatterplot(data[:, 3], data[:, 2], hue=data[:,-1])

# plt.hlines(potential_splits[2], xmin=0, xmax=2.5)
# plt.show()

## Spliting the dataset in two groups i.e. left and right subtree

In [ ]:
# split column is the feature by which we are splittting a node
# split_value is the decision to be made at a node 
#  Make two data sets (data_below, data_above) going to the left and the right subtree of a node

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    feature_type = FEATURE_TYPES[split_column]
    if feature_type == 'continuous':
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values !=  split_value]
    return data_below, data_above

## Calculate entropy if a split

In [ ]:
def calculate_entropy(data):
    
    vals, counts = np.unique(data[:, -1], return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

## Calculate total entropy (average weighted entropy)

In [ ]:
def calculate_overall_entropy(data, data_below, data_above):
    
    prob_data_below = len(data_below) / len(data)
    prob_data_above = len(data_above) / len(data)

    overall_entropy =  (prob_data_below * calculate_entropy(data_below) 
                      + prob_data_above * calculate_entropy(data_above))
    
    return overall_entropy

## Find best split (one with least entropy)

For each potential split of every column, find entropy

In [ ]:
def determine_best_split(data, potential_splits):
    
    min_entropy = 9999 # initial big value like INT_MAX

    for column_index in potential_splits: 
        for value in potential_splits[column_index]:
    
            data_below, data_above = split_data(data, column_index, value)
            current_overall_entropy = calculate_overall_entropy(data, data_below, data_above)

            if current_overall_entropy <= min_entropy:
                min_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Representation of decision tree

In [ ]:
# sub_tree = {"question": ["yes_answer", 
#                          "no_answer"]}

In [ ]:
# example_tree = {"petal_width <= 0.8": ["Iris-setosa", 
#                                       {"petal_width <= 1.65": [{"petal_length <= 4.9": ["Iris-versicolor", 
#                                                                                         "Iris-virginica"]}, 
#                                                                                         "Iris-virginica"]}]}

## Determine feature type

In [ ]:
def determine_feature_type(df):
    
    feature_types = []
    unique_treshold = 15
    for feature in df.columns[:-1]:
      unique_values = df[feature].unique()
      example_value = unique_values[0]

      if (isinstance(example_value, str)) or (len(unique_values) <= unique_treshold):
          feature_types.append("categorical")
      else:
          feature_types.append("continuous")
    
    return feature_types

## Decision Tree Classifier Algorithm

In [ ]:
def decision_tree_classifier(df, counter=0, min_samples=2, max_depth=5):
    
    if counter == 0:
      global FEATURE_TYPES, FEATURE_NAMES
      FEATURE_NAMES = df.columns
      FEATURE_TYPES = determine_feature_type(df)
      data = df.values
    else:
      data = df           
    
    
    # base case
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
      classification = classify_data(data)
      return classification

    else:    
      counter += 1

      potential_splits = get_potential_splits(data)
      split_column, split_value = determine_best_split(data, potential_splits)
      data_below, data_above = split_data(data, split_column, split_value)
      
      feature_name = FEATURE_NAMES[split_column]
      
      feature_type = FEATURE_TYPES[split_column]
      if feature_type == 'continuous':
        question = "{} <= {}".format(feature_name, split_value)
      else:
        question = "{} = {}".format(feature_name, split_value)

      sub_tree = {question: []}
      
      yes_ans = decision_tree_classifier(data_below, counter, min_samples, max_depth)
      no_ans = decision_tree_classifier(data_above, counter, min_samples, max_depth)
      
      if yes_ans == no_ans:
          sub_tree = yes_ans
      else:
          sub_tree[question].append(yes_ans)
          sub_tree[question].append(no_ans)
      
      return sub_tree

## Classification

In [ ]:
def classify_example(example, dtc):
    question = list(dtc.keys())[0]
    feature_name, comparison_operator, value = question.split()

    # question
    if comparison_operator == "<=":
      if example[feature_name] <= float(value):
          answer = dtc[question][0] # yes_ans
      else:
          answer = dtc[question][1] # no_ans
    else:
      if str(example[feature_name]) == value:
          answer = dtc[question][0]
      else:
          answer = dtc[question][1]
    # base case
    if not isinstance(answer, dict):
        return answer
    
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [ ]:
def calculate_accuracy(df, dtc):

    df["classification"] = df.apply(classify_example, axis=1, args=(dtc,))
    df["classification_correct"] = df["classification"] == df["target"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2)

dt_classifier = decision_tree_classifier(train_data, max_depth=3)

accuracy = calculate_accuracy(test_data, dt_classifier)
print(accuracy)

pprint(dt_classifier)
test_data.loc[test_data['classification']!=test_data['target']]

0.9666666666666667
{'petal_width <= 0.6': ['setosa',
                        {'petal_length <= 4.9': [{'petal_width <= 1.6': ['versicolor',
                                                                         'virginica']},
                                                 'virginica']}]}


,sepal_length,sepal_width,petal_length,petal_width,target,classification,classification_correct
77,6.7,3.0,5.0,1.7,versicolor,virginica,False


## Running on a dataset with categorical variables

In [ ]:
df2 = sns.load_dataset('titanic')
df2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
df2['target'] = df2['survived']
df2.drop(columns=df2.columns[8:-1].tolist()+['survived'], axis=1, inplace=True)

In [ ]:
df2.head()

,pclass,sex,age,sibsp,parch,fare,embarked,target
0,3,male,22.0,1,0,7.2500,S,0
1,1,female,38.0,1,0,71.2833,C,1
2,3,female,26.0,0,0,7.9250,S,1
3,1,female,35.0,1,0,53.1000,S,1
4,3,male,35.0,0,0,8.0500,S,0


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    891 non-null    int64  
 1   sex       891 non-null    object 
 2   age       714 non-null    float64
 3   sibsp     891 non-null    int64  
 4   parch     891 non-null    int64  
 5   fare      891 non-null    float64
 6   embarked  889 non-null    object 
 7   target    891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [ ]:
df2.fillna({'age': df2['age'].median(),
           'embarked': df2['embarked'].mode()[0]
          }, inplace = True)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    891 non-null    int64  
 1   sex       891 non-null    object 
 2   age       891 non-null    float64
 3   sibsp     891 non-null    int64  
 4   parch     891 non-null    int64  
 5   fare      891 non-null    float64
 6   embarked  891 non-null    object 
 7   target    891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [ ]:
train_data2, test_data2 = train_test_split(df2, test_size=0.2)

dtc2 = decision_tree_classifier(train_data2, max_depth=3)

accuracy2 = calculate_accuracy(test_data2, dtc2)
print(accuracy2)

pprint(dtc2, width=50)
# test_data.loc[test_data['classification']!=test_data['target']]

0.7865168539325843
{'sex = male': [{'fare <= 26.0': [{'age <= 9.0': [1,
                                                  0]},
                                  {'fare <= 26.3875': [1,
                                                       0]}]},
                {'pclass = 3': [{'fare <= 23.25': [1,
                                                   0]},
                                1]}]}
